In [1]:
import pandas as pd

In [3]:
import pandas as pd

df = pd.read_csv('toothgrowth.csv')


In [4]:
df

,len,supp,dose
0,4.2,VC,0.5
1,11.5,VC,0.5
2,7.3,VC,0.5
3,5.8,VC,0.5
4,6.4,VC,0.5
5,10.0,VC,0.5
6,11.2,VC,0.5
7,11.2,VC,0.5
8,5.2,VC,0.5
9,7.0,VC,0.5


In [6]:
grouped_mean = df.groupby('supp')['len'].mean()
grouped_mean

supp
OJ    20.663333
VC    16.963333
Name: len, dtype: float64

- 귀무가설 : 오렌지주스를 투여받은 기니피그의 치아 길이 평균은 아스코르브산을 투여받은 기니피그의 치아 길이 평균과 같다.
- 대립가설 : 오렌지주스를 투여받은 기니피그의 치아 길이 평균은 아스코르브산을 투여받은 기니피그의 치아 길이 평균과 다르다.

In [12]:
# 정규성 검정
vc_group = df[df['supp'] == 'VC']['len']
oj_group = df[df['supp'] == 'OJ']['len']

_, p_value_vc = shapiro(vc_group)
_, p_value_oj = shapiro(oj_group)

print(f"VC Group's Normality Test p-value: {p_value_vc}")
print(f"OJ Group's Normality Test p-value: {p_value_oj}")

# 등분산성 검정
_, p_value_levene = levene(vc_group, oj_group)

print(f"Levene's Test p-value: {p_value_levene}")


VC Group's Normality Test p-value: 0.4284436106681824
OJ Group's Normality Test p-value: 0.023587485775351524
Levene's Test p-value: 0.2751764616144052


정규성 검정:

VC Group: p-value가 0.4284로, 0.05보다 크다. 따라서 VC 그룹의 데이터는 정규분포를 따른다고 볼 수 있다.
OJ Group: p-value가 0.0236으로, 0.05보다 작다. 이는 OJ 그룹의 데이터가 정규분포를 따르지 않을 가능성이 높다는 것을 의미한다.
등분산성 검정:

p-value가 0.2752로, 0.05보다 크다. 따라서 두 그룹(VC와 OJ)의 분산은 동일하다고 볼 수 있다.
종합해석:

VC 그룹의 데이터는 정규분포를 따르는 반면, OJ 그룹의 데이터는 정규분포를 따르지 않을 가능성이 높다.
두 그룹의 분산은 동일하다고 볼 수 있다.

## OJ 그룹이 정규분포를 따르지 않는 것으로 확인되었으므로, 데이터를 정규화하기 위해  Box-Cox 변환을 사용

In [9]:
from scipy.stats import boxcox

# OJ 그룹 데이터 추출
oj_group = df[df['supp'] == 'OJ']['len']

# Box-Cox 변환
oj_transformed, _ = boxcox(oj_group)

# 변환된 데이터의 정규성 검정
_, p_value_oj_transformed = shapiro(oj_transformed)

print(f"Transformed OJ Group's Normality Test p-value: {p_value_oj_transformed}")


Transformed OJ Group's Normality Test p-value: 0.10078078508377075


In [10]:
from scipy.stats import ttest_ind

# VC 그룹과 OJ 그룹 데이터 추출
vc_group = df[df['supp'] == 'VC']['len']
oj_group = df[df['supp'] == 'OJ']['len']

# 독립 표본 t-검정
t_stat, p_value = ttest_ind(vc_group, oj_group)

print(f"T-statistic: {t_stat}")
print(f"P-value: {p_value}")


T-statistic: -1.91526826869527
P-value: 0.06039337122412848


귀무가설을 채택하고, 0.05의 유의 수준에서 VC 그룹과 OJ 그룹 간의 치아 길이에 대한 유의한 차이는 발견되지 않았다.